### Enter variables

In [2]:
# Input files
storage_shapefile = "OLF_Storages_RobClean.shp"
storage_id_field = "Wks_Num_tx"

# Output files
outputCSV = "GEE_Map_Landsat.csv"

# Cloud cover
CLOUD_FILTER = 40 # Under 40%

# Thresholds
ndwiMaskVal = -0.1

# UTM Zone
utmZoneInfo = 'EPSG:28355'#z55S

# Dates
start_date = '1986-01-01'
end_date = '2023-02-22'

### Import modules

In [3]:
import ee, geemap, folium, pandas, geopandas as gpd, json, datetime, altair as alt, csv
from IPython.display import Image
import ee.mapclient
import datetime
import os
from os.path import exists as file_exists

import requests

# importing pandas as pd
import pandas as pd

ModuleNotFoundError: No module named 'ee'

### Declare functions

In [50]:
def test_api():
    dem = ee.Image('USGS/SRTMGL1_003')
    xy = ee.Geometry.Point([86.9250, 27.9881])
    elev = dem.sample(xy, 30).first().get('elevation').getInfo()
    print('Authentication active')

#NDVI calculation:
def areadate(img):
    area = img.gt(0).multiply(ee.Image.pixelArea()).divide(1000000).reduceRegion(ee.Reducer.sum(), storage_geometry, 30).get('NDWI');
    return img.set('area', area).set('date', img.get('system:time_start'));

def ndwi_func(i):
    ndwi = i.normalizedDifference (['green', 'swir']).rename('NDWI')
    return i.addBands(ee.Image(ndwi))

def ndvi_func(i):
    ndvi = i.normalizedDifference (['red', 'swir']).rename('NDVI')
    return i.addBands(ee.Image(ndvi))

def mndwi_func(i):
    mndwi = i.normalizedDifference (['nir', 'swir']).rename('MNDWI')
    return i.addBands(ee.Image(mndwi))

def addDate(image):
    # parse date stored in 'system:index'
    date = ee.Date(image.get('system:index'))

    # format date, see http:#www.joda.org/joda-time/apidocs/org/joda/time/format/DateTimeFormat.html
    str = date.format('YYYY-mm-dd')

    return image.set({'Date': str})

def mean_func(image):
    mean = (image                        
            #.mean()
            .reduceRegion(ee.Reducer.mean()))#, storage_geometry, 30))
    mean = mean.getInfo()            
    mean = mean.get('nd')    
    return mean

def year_func(y):
    range = ee.Filter.calendarRange (y, y, 'year')
    mean = (l57ndwi
            .filter(range)
            .select('NDWI')
            .mean()
            .reduceRegion(ee.Reducer.mean(), storage_geometry, 30))
    return mean.set('year', y)

def year_indices(y):
    range = ee.Filter.calendarRange (y, y, 'year')
    ndwimean = (images
        .select('NDWI')
        .filter(range)
        .mean()
        .reduceRegion(ee.Reducer.mean(), storage_geometry, 30))        
    ndwimean = ndwimean.getInfo()

    ndvimean = (images
        .select('NDVI')
        .filter(range)
        .mean()
        .reduceRegion(ee.Reducer.mean(), storage_geometry, 30))        
    ndvimean = ndwimean.getInfo()

    ndwimean = (images
        .select('MNDWI')
        .filter(range)
        .mean()
        .reduceRegion(ee.Reducer.mean(), storage_geometry, 30))        
    mndwimean = ndwimean.getInfo()

    results = [worksNumber,y,ndwimean,ndvimean,mndwimean]
    storageStats.loc[len(storageStats)] = results 
    print(results)
    
def get_area(img):
    total_area = img.multiply(ee.Image.pixelArea())
    ndwi = img.gte(ndwiMaskVal).rename('ndwiMaskLayer')
    ndwi_area = ndwi.multiply(ee.Image.pixelArea())
    return img.set('total_area', total_area).set('ndwi_area', ndwi_area)

def test_api():
    dem = ee.Image('USGS/SRTMGL1_003')
    xy = ee.Geometry.Point([86.9250, 27.9881])
    elev = dem.sample(xy, 30).first().get('elevation').getInfo()
    print('Authentication active')
    
def transformer(feature):
    transformed_feature = feature.transform(utmZoneInfo, 0.001);
    return transformed_feature;

#def get_area(feature):
#    area = feature.area()
#    area = ee.Number(area).divide(1e6).round()
#    return area

def extract_water(img):
    ndwi_image = img.normalizedDifference(['B2', 'B5'])
    water_image = ndwi_image.gt(ndwiMaskVal)
    return water_image

def get_stats(image):
    date = ee.Date(image.get('system:time_start'))        
    date = date.format('dd-MM-YYYY')
    allPixels = image.get('allPixels')
    allPixels = allPixels.getInfo()
    ndwiPixels = image.get('ndwiPixels')
    ndwiPixels = ndwiPixels.getInfo()
    CLOUD_COVER = image.get('CLOUD_COVER')
    CLOUD_COVER = CLOUD_COVER.getInfo()
    return date, allPixels, ndwiPixels, CLOUD_COVER

def rename_bands(bands):
    image.select(bands,STD_NAMES)
    return image
    

def filterImages(storage_geometry):
    # Get band names
    S2_BANDS = ['B2',   'B3',    'B4',  'B8',  'B11']; #Sentinel 2
    LC8_BANDS = ['B2',   'B3',    'B4',  'B5',  'B6']; #Landsat 8
    LC457_BANDS = ['B1',   'B2',    'B3',  'B4',  'B5']; #Landsat 4,5,7
    STD_NAMES = ['blue', 'green', 'red', 'nir', 'swir'] # Standard names
    
    # Load Landsat 5, 7, 8
    s2 = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(storage_geometry).select(['B2','B3','B4','B8','B11'])
    l4 = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2").filterBounds(storage_geometry).select(['B1','B2','B3','B4','B5'])
    l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").filterBounds(storage_geometry).select(['B1','B2','B3','B4','B5'])
    l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filterBounds(storage_geometry).select(['B1','B2','B3','B4','B5'])
    l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(storage_geometry).select(['B2','B3','B4','B5','B6'])
    
    # Rename bands
    s2 = s2.map(lambda image: image.select(S2_BANDS, STD_NAMES))
    l4 = l4.map(lambda image: image.select(LC457_BANDS, STD_NAMES))
    l5 = l5.map(lambda image: image.select(LC457_BANDS, STD_NAMES))
    l7 = l7.map(lambda image: image.select(LC457_BANDS, STD_NAMES))
    l8 = l8.map(lambda image: image.select(LC8_BANDS, STD_NAMES))
    
    # Filter images by cloud cover
    s2 = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
    l8 = l8.filter(ee.Filter.lt('CLOUD_COVER', CLOUD_FILTER))
    l7 =l7.filter(ee.Filter.lt('CLOUD_COVER', CLOUD_FILTER))
    l5 = l5.filter(ee.Filter.lt('CLOUD_COVER', CLOUD_FILTER))
    l4 = l4.filter(ee.Filter.lt('CLOUD_COVER', CLOUD_FILTER))
    
    # Merge collections
    images = l4.merge(l5).merge(l7).merge(l8).merge(s2)
    # Clip images to storage
    images = images.map(lambda image: image.clip(storage_geometry).reproject(crs=utmZoneInfo))
    #.filterDate(start_date, end_date)
    # Add date
    images.map(addDate)
    # Sort by date
    images = images.sort('Date')
    # Add indices
    images.map(ndwi_func).map(ndvi_func).map(mndwi_func)
    return images

def exportStats(worksNumber):
    # Create a new dataframe
    storageStats = pandas.DataFrame(columns=['Works Number', 'Year', 'NDWI', 'NDVI', 'MNDWI'])        
    # Select the feature
    storage_selection = storage_collection.filter(ee.Filter.eq("Wks_Num_tx", worksNumber))   
    # Get the geometry
    storage_geometry = storage_selection.geometry()
    # Get the area of the storage
    storage_area = storage_geometry.area(1).getInfo()
    # Create list of images
    images = filterImages(storage_geometry)           
    # Create a list of images
    imageSize = images.size().getInfo()    
    imageList = images.toList(images.size())
    # Get stats ##
    # For each image
    for i in range(0, imageSize):    
        # Choose the image
        image = ee.Image(imageList.get(i))
        # Get date
        date = ee.Date(image.get('system:time_start')).format("yyyy-MM-dd")
        date = date.getInfo()
        # Create images
        ndvi_image = image.select("NDVI")
        ndwi_image = image.select("NDWI")
        mndwi_image = image.select("MNDWI")
        #ndvi_image = image.normalizedDifference(['B4', 'B3'])
        #ndwi_image = image.normalizedDifference(['B4', 'B5'])
        #mndwi_image = image.normalizedDifference(['B2', 'B5'])
        # Get the area of the storage
        storage_area = storage_geometry.area(1).getInfo()
        # Get water area - Mask water areas using MNDWI
        water_image = mndwi_image.gt(ndwiMaskVal).selfMask()
        water_vector = water_image.reduceToVectors()
        water_area = water_vector.geometry().area(1).getInfo()
        # Get average ndvi, ndwi and mndwi values            
        ndvi = mean_func(ndvi_image)
        ndwi = mean_func(ndwi_image)
        mndwi = mean_func(mndwi_image)
        # Get cloud cover
        cloudiness = image.get("CLOUD_COVER").getInfo()
        # Add results to storage stats dataframe
        results = [worksNumber,date,storage_area,water_area,ndvi,ndwi,mndwi,cloudiness]
        print(results)
        storageStats.loc[len(storageStats)] = results   
        # Export image
        #exportRGBImage(image)
   
    # Save to CSV
    storageStats.to_csv(outputCSV)

def daily_stats(image):
    # Get date
    date = ee.Date(image.get('system:time_start')).format("yyyy-MM-dd")
    date = date.getInfo()
    # Create images
    ndvi_image = image.select('NDVI')
    ndwi_image = image.select('NDWI')
    mndwi_image = image.select('MNDWI')
    # Get water area - Mask water areas using MNDWI
    water_image = mndwi_image.gt(ndwiMaskVal).selfMask()
    water_vector = water_image.reduceToVectors()
    water_area = water_vector.geometry().area(1).getInfo()
    # Get average ndvi, ndwi and mndwi values            
    ndvi = mean_func(ndvi_image)
    ndwi = mean_func(ndwi_image)
    mndwi = mean_func(mndwi_image)
    # Get cloud cover
    try:
        cloudiness = image.get("CLOUD_COVER").getInfo()
    except:
        cloudiness = image.get("CLOUDY_PIXEL_PERCENTAGE").getInfo()
    # Add results to storage stats dataframe
    results = [worksNumber,date,storage_area,water_area,ndvi,ndwi,mndwi,cloudiness]
    return results
    #print(results)
    #storageStats.loc[len(storageStats)] = results   
    # Export image
    #exportRGBImage(image)
    
    
def exportRGBImage(worksNumber):
    # Select the feature
    storage_selection = storage_collection.filter(ee.Filter.eq("Wks_Num_tx", worksNumber))   
    # Get the geometry
    storage_geometry = storage_selection.geometry()
    storage_geometry = storage_geometry.buffer(distance=500)
    storage_geometry = storage_geometry.bounds()
    # Get the latest image
    s2 = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(storage_geometry).select(['B2','B3','B4'])
    # Filter images by cloud cover
    s2 = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))
    # Get the latest image
    imageSize = s2.size().getInfo()    
    imageList = s2.toList(s2.size())
    # Choose the image
    image = ee.Image(imageList.get(imageSize-1))        
    # Create a RGB image for export
    stats = image.reduceRegion(reducer=ee.Reducer.minMax(),geometry=storage_geometry,scale=100,bestEffort=True)
    statDict = stats.getInfo()
    try: 
        prettyImg = image.visualize(bands=['B4', 'B3', 'B2'],                                 
             min=[statDict['B4_min'],statDict['B3_min'],statDict['B2_min']],
             max=[statDict['B4_max'],statDict['B3_max'],statDict['B2_max']],
             gamma=1)
        
        # Export multi-band GeoTIFF file.
        url = prettyImg.getDownloadUrl({
            #'bands': ['B4', 'B3', 'B2'],
            'region': storage_geometry,
            #'scale': 10,
            'format': 'GEO_TIFF'
        })            
        response = requests.get(url)
        with open(filename, 'wb') as fd:                
          fd.write(response.content)
    except:
        print('Export failed')

    return

def exportMap(worksNumber):
    # Select the feature
    storage_selection = storage_collection.filter(ee.Filter.eq("Wks_Num_tx", worksNumber))   
    # Get the geometry
    storage_geometry = storage_selection.geometry()
    storage_geometry = storage_geometry.buffer(distance=500)
    storage_geometry = storage_geometry.bounds()
    # Get the latest image
    s2 = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(storage_geometry).select(['B2','B3','B4'])
    # Filter images by cloud cover
    s2 = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))
    # Get the latest image
    imageSize = s2.size().getInfo()    
    imageList = s2.toList(s2.size())
    # Choose the image
    image = ee.Image(imageList.get(imageSize-1))        
    # Create a RGB image for export
    stats = image.reduceRegion(reducer=ee.Reducer.minMax(),geometry=storage_geometry,scale=100,bestEffort=True)
    statDict = stats.getInfo()
    try: 
        prettyImg = image.visualize(bands=['B4', 'B3', 'B2'],                                 
             min=[statDict['B4_min'],statDict['B3_min'],statDict['B2_min']],
             max=[statDict['B4_max'],statDict['B3_max'],statDict['B2_max']],
             gamma=1)
        
        # Add to map
        Map = geemap.Map()
        Map.centerObject(prettyImg, zoom=8)
        Map.addLayer(prettyImg, {}, "Sentinel 2")
        ee_object = geemap.geojson_to_ee(storage_selection)
        Map.addLayer(ee_object, {}, 'Storage')
        Map.to_image(filename=filename, monitor=1)
                     
    except:
        print('Export failed')

    return

### Authenticate Earth Engine

In [51]:
# Test access then authenticate if fails
try:
    test_api()
except:    
    ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

Authentication active


### Convert shapefile into feature collection

In [52]:
# Load shapefile into geopandas
storage_shapes = gpd.read_file(storage_shapefile)

# Convert shapes into json
json_storages = json.loads(storage_shapes.to_json())

# Convert json into feature collection
storage_collection =  ee.FeatureCollection(json_storages)

# Reproject collection
storage_collection = storage_collection.map(transformer);

# Add area
storage_collection.map(lambda feature: feature.set('total_area', ee.Number(feature.area()).divide(1e6).round()))

### Get a list of Storage IDs

In [53]:
# list the unique quality code values
worksSet = sorted(storage_shapes[storage_id_field].unique())
print(worksSet)

['14688', '14916', '14923', '17563', '17879', '17883', '17890', '18215', '19819', '19823', '19830', '19883', '19921', '19939', '19941', '19967', '19974', '19978', '20553', '20554', '20559', '20560', '20627', '20657', '20658', '20662', '20664', '20665', '20668', '20669', '21462', '21515', '21516', '21517', '21518', '21519', '21520', '21521', '21549', '21557', '21559', '21562', '21564', '21565', '21566', '21572', '21577', '21624', '21626', '21631', '21634', '21635', '21636', '21649', '21650', '21656', '21717', '21719', '23743', '23744', '23745', '24064', '24080', '24082', '24084', '24085', '24090', '24091', '24673', '24674', '24675', '24676', '24677', '24685', '24686', '24689', '24690', '24752', '24753', '24761', '24762', '24765', '24780', '24781', '24782', '24977', '24978', '24985', '24998', '25000', '25004', '25010', '25023', '25025', '25027', '25028', '25031', '25034', '25041', '25047', '25058', '25071', '25093', '25103', '25104', '25105', '25106', '25108', '25110', '25111', '25112', 

### Clip imagery to each storage

In [11]:
# Start timer
startNow = datetime.datetime.now()
print("Starting at " + str(startNow))

# For each storage
for worksNumber in worksSet:
    
    # Create a image filename
    filename = "output\\" + worksNumber + ".png"
    
    # Check if already completed
    if file_exists(filename):
        
        print("Skipping: " + filename)
        
    else:
    
        # Start process
        print("Exporting: " + filename)

        # Create an image function to map
        exportMap(worksNumber)

# Print the storage stats
print("Finished")

# Finish time
endNow = datetime.datetime.now()
diff = (endNow-startNow).total_seconds()
print("End at " + str(endNow))
print("Taking: " + str(diff/60) + " minutes")

Starting at 2022-09-19 12:21:19.611838
Skipping: output\14688.png
Skipping: output\14916.png
Skipping: output\14923.png
Exporting: output\17563.png
Skipping: output\17879.png
Skipping: output\17883.png
Skipping: output\17890.png
Skipping: output\18215.png
Skipping: output\19819.png
Exporting: output\19823.png


KeyboardInterrupt: 

### Open image in Geopandas and add boundary

In [54]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [55]:
# Select the feature
worksNumber = '14688'
storage_selection = storage_collection.filter(ee.Filter.eq("Wks_Num_tx", worksNumber))   
# Get the geometry
storage_geometry2 = storage_selection.geometry()
storage_geometry = storage_selection.geometry()
storage_geometry = storage_geometry.buffer(distance=500)
storage_geometry = storage_geometry.bounds()
# Get the latest image
s2 = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(storage_geometry).select(['B2','B3','B4'])
# Filter images by cloud cover
s2 = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))
# Get the latest image
imageSize = s2.size().getInfo()    
imageList = s2.toList(s2.size())
# Choose the image
image = ee.Image(imageList.get(imageSize-1))        
# Create a RGB image for export
stats = image.reduceRegion(reducer=ee.Reducer.minMax(),geometry=storage_geometry,scale=100,bestEffort=True)
statDict = stats.getInfo()

prettyImg = image.visualize(bands=['B4', 'B3', 'B2'],                                 
     min=[statDict['B4_min'],statDict['B3_min'],statDict['B2_min']],
     max=[statDict['B4_max'],statDict['B3_max'],statDict['B2_max']],
     gamma=1)




In [56]:
# Add to map
Map = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [57]:
#Map.centerObject(prettyImg, zoom=8)
#Map.addLayer(prettyImg, {}, "Sentinel 2")
#ee_object = geemap.geojson_to_ee(storage_selection)
Map.addLayer(image, {}, 'Storage')
#Map.to_image(filename=filename, monitor=1)

In [58]:
Map

Map(bottom=754.0, center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [59]:
Map.centerObject(prettyImg, zoom=1)
Map.addLayer(prettyImg, {}, "Sentinel 2")

In [60]:
Map

Map(bottom=756.278848, center=[-28.511135979585635, 148.58267874484523], controls=(WidgetControl(options=['pos…